it was pretrained with three objectives:

* Distillation loss: the model was trained to return the same probabilities as the BERT base model.
* Masked language modeling (MLM): this is part of the original training loss of the BERT base model. When taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally mask the future tokens. It allows the model to learn a bidirectional representation of the sentence.
* Cosine embedding loss: the model was also trained to generate hidden states as close as possible as the BERT base model.

In [10]:
from transformers import DistilBertTokenizer, DistilBertModel

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
print(tokenizer.model_input_names)
print(tokenizer.all_special_tokens)

['input_ids', 'attention_mask']
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [12]:
model = DistilBertModel.from_pretrained("bert-base-uncased")
model

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing DistilBertModel: ['bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.11.attention.self.key.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.6.attention.output.dense.weight', 'bert.encoder.layer.4.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.10.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.self.key.weight', 'bert.encoder.layer.2.attention.self.query.weight', 'bert.encoder.layer.4.attention.self.query.weight', 'bert.encoder.layer.5.attention.self.key.bias', 'bert.encoder.layer.6.attention.self.value.weight', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.4.attention.sel

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [13]:
model.config

DistilBertConfig {
  "_name_or_path": "bert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [14]:
for name, para in model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
transformer.layer.0.attention.q_lin.weight
transformer.layer.0.attention.q_lin.bias
transformer.layer.0.attention.k_lin.weight
transformer.layer.0.attention.k_lin.bias
transformer.layer.0.attention.v_lin.weight
transformer.layer.0.attention.v_lin.bias
transformer.layer.0.attention.out_lin.weight
transformer.layer.0.attention.out_lin.bias
transformer.layer.0.sa_layer_norm.weight
transformer.layer.0.sa_layer_norm.bias
transformer.layer.0.ffn.lin1.weight
transformer.layer.0.ffn.lin1.bias
transformer.layer.0.ffn.lin2.weight
transformer.layer.0.ffn.lin2.bias
transformer.layer.0.output_layer_norm.weight
transformer.layer.0.output_layer_norm.bias
transformer.layer.1.attention.q_lin.weight
transformer.layer.1.attention.q_lin.bias
transformer.layer.1.attention.k_lin.weight
transformer.layer.1.attention.k_lin.bias
transformer.layer.1.attention.v_lin.weight
transformer.lay

In [15]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
print(encoded_input.keys())
print(encoded_input['input_ids'].shape)  # [1, 12]

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
dict_keys(['input_ids', 'attention_mask'])
torch.Size([1, 12])


In [16]:
result = model(**encoded_input)
print(result.last_hidden_state.shape)  # # [1, 12, 768](限制input_ids最大长度为512)
print(result.last_hidden_state)



torch.Size([1, 12, 768])
tensor([[[ 0.7310,  1.0497,  1.0116,  ...,  1.4013, -2.2519,  1.4625],
         [ 0.4230, -0.5819, -0.6813,  ...,  0.6490, -1.5736,  1.3549],
         [ 0.6843,  0.8295,  0.5275,  ..., -0.5573, -0.3812,  1.5138],
         ...,
         [ 0.3765,  0.5286,  0.8991,  ..., -0.1424, -0.7444,  1.7031],
         [ 0.6490, -1.4208,  0.8058,  ..., -0.7703, -0.8962,  0.6566],
         [ 0.2249,  0.5368,  1.2544,  ..., -0.5541,  0.7447,  1.2399]]],
       grad_fn=<NativeLayerNormBackward0>)
